# Module 4 - Operational Commands

## Overview

In the last section you created an Address object. This is a configuration change you made by creating a config tree in memory that represents what you want, then pushing it to the firewall. We'll do more with configuration in the next module, but for now let's explore other features and tools of Device Framework.

Operational commands can be executed on the firewall to get information on the state of the device. Operational commands are sometimes nicknamed as 'show commands' and can be executed on the CLI or the API. Operational commands do not change the configuration.

One of the most common operational commands is 'show system info' which gives information about the firewall's hardware platform and software version. Run the following cell to call 'show system info' on the firewall.

In [2]:
# Start by importing the modules and creating a Firewall object
from pandevice import firewall
fw = firewall.Firewall('10.30.11.101', 'admin', 'Ignite18')

In [3]:
# This is the 'fw' variable we created in the last section
# The op method called on the firewall invokes an operational command
fw.op('show system info')

<Element 'response' at 0x7fcdf4bdecc8>

You probably got a response that looks something like this: `<Element 'response' at 0x7f49a84464f8>`

What is this? It's an XML response from the firewall but it doesn't look like XML because it's packaged into a python object. Device Framework assumes you'll be looking for something specific in this XML response, so it parses the XML for you and returns it as an Element object. An Element object is part of the python standard [ElementTree library](https://docs.python.org/3/library/xml.etree.elementtree.html).

If you know the structure of the XML response and where the thing you're looking for is, you can find it with an XPath:

In [4]:
# First, call the op command and store the Element response
element_response = fw.op('show system info')

In [5]:
# Now, find the XPath for the firewall's hostname in the XML and get its text
element_response.find('./result/system/hostname').text

'PA-VM'

In [6]:
# Same for the management IP of the firewall, also part of 'show system info'
element_response.find('./result/system/ip-address').text

'10.30.11.101'

## Response as XML

If you want to see the whole XML response, import the ElementTree library and turn the Element object into an XML string using its `tostring()` method:

In [7]:
import xml.etree.ElementTree as ET
ET.tostring(fw.op('show system info'))

b'<response status="success"><result><system><hostname>PA-VM</hostname><ip-address>10.30.11.101</ip-address><public-ip-address>unknown</public-ip-address><netmask>255.255.255.0</netmask><default-gateway>10.30.11.254</default-gateway><is-dhcp>no</is-dhcp><ipv6-address>unknown</ipv6-address><ipv6-link-local-address>fe80::20c:29ff:fe06:96b0/64</ipv6-link-local-address><ipv6-default-gateway /><mac-address>00:0c:29:06:96:b0</mac-address><time>Thu May  3 14:44:21 2018\n</time>\n<uptime>0 days, 1:30:46</uptime>\n<devicename>PA-VM</devicename>\n<family>vm</family><model>PA-VM</model><serial>015351000001577</serial><vm-mac-base>D4:F4:BE:D6:09:00</vm-mac-base><vm-mac-count>256</vm-mac-count><vm-uuid>564D2137-C7C2-C361-655C-F1083F0696B4</vm-uuid><vm-cpuid>ESX:FB060000FFFB8B0F</vm-cpuid><vm-license>VM-50</vm-license><vm-mode>VMWare ESXi</vm-mode>\n<cloud-mode>non-cloud</cloud-mode>\n<sw-version>8.1.0</sw-version>\n<global-protect-client-package-version>0.0.0</global-protect-client-package-version>

## Operational Convenience Methods

Some operational commands like 'show system info' are so commonly used that Device Framework has convenience methods for them. The conveniance method for 'show system info' is `refresh_system_info()`.

This method doesn't just pull the system info, it stores some of the relevant information in the Firewall object.  Let's try it:

In [8]:
# This will return None for each variable because they are not known yet
(fw.version, fw.platform, fw.serial)

(None, None, None)

In [9]:
fw.refresh_system_info();

In [10]:
(fw.version, fw.platform, fw.serial)

('8.1.0', 'PA-VM', '015351000001577')

Notice that before `refresh_system_info()` the variables are `None`, but after `refresh_system_info()` the variables are populated with the current state of the firewall. The word **refresh** has a special meaning in Device Framework: pull information from the live device and store it in the object. In this case, system info is pulled from the live firewall and stored in the Firewall object.